In [1]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

In [2]:
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import load_tools
from pydantic import BaseModel, Field

llm = ChatOpenAI(temperature=0)

search = SerpAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm)


class CalculatorInput(BaseModel):
    question: str = Field(description="The math question to answer.")


tools = [
    Tool.from_function(
        func=search.run,
        name = "Search",
        description="useful for when you need to answer questions about current events"
        # coroutine= ... <- you can specify an async method if desired as well
    ),
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="useful for when you need to answer questions about math",
        args_schema=CalculatorInput
        # coroutine= ... <- you can specify an async method if desired as well
    ),
    *load_tools([
        "python_repl",
        "requests",
        "terminal",
    ])
]

/Users/toubatbrian/Desktop/langchain/langchain/agents/load_tools.py:405: UserWarning: tool name `requests` is deprecated - please use `requests_all` or specify the requests method
  warnings.warn(


In [3]:
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.experimental.autoflow.schema import Schema, FlowSchemaOutputParser, StringSchema
from langchain import PromptTemplate
from langchain.experimental.autoflow.prompts import TASK_BREAKDOWN

schema = Schema.object(
    solvability_thought=Schema.string("The thought process you used to determine if the task is solvable"),
    solvable=Schema.boolean("Whether the task is solvable"),
    plan=Schema.string("- short bulleted\n- list that conveys\n- long-term plan on how to solve main task step by step"),
    reflection=Schema.string("Your reflection on the plan above, any advice to improve"),
    plan_score=Schema.number("From 1 to 10, score your plan from 1 (completely irrelavant and must improve) to 10 (perfect plan, no improvement needed")
)

llm = ChatOpenAI(temperature=0, model="gpt-4")
output_parser = FlowSchemaOutputParser(flow_schema=schema)

prompt = PromptTemplate.from_template(
    TASK_BREAKDOWN,
    template_format="jinja2",
    output_parser=output_parser
)

llm_chain = LLMChain(
    llm=llm, 
    prompt=prompt, 
    verbose=True
)

In [4]:
attempt_history = []
while True:
    try:
        result = llm_chain.predict_and_parse(
            task="Based on input topic, search for relevant information, calculate the length of the information, and write the <info_string>:<char_count> into a file, output file location",
            input_schema=Schema.object(
                topic=Schema.string("selected topic")
            ),
            output_schema=Schema.object(
                location=Schema.string("file location")
            ),
            attempt_history=attempt_history,
            format_instructions=output_parser.get_format_instructions(),
            tools=tools,
            len=len
        )
    except:
        print(result)
        break
    
    if not result['solvable']:
        print("Not solvable")
        break

    if result['plan_score'] >= 6:
        break
    
    attempt_history.append(result)
    
result



> Entering new LLMChain chain...
Prompt after formatting:
You are acted as a task breakdown expert. Your job is to breakdown a task into a list of subtasks that potentially can be solved by existing tools.
Subtasks form a dependency flow graph that solve the main task, so that one subtask may depends on the output of other subtasks.

[TOOL_DESCRIPTION]
You have access to a set of tools with their description:

- Search: useful for when you need to answer questions about current events
    - input_schema: {'tool_input': {'type': 'string'}}
    - output_schema: { "result": string }

- Calculator: useful for when you need to answer questions about math
    - input_schema: {'question': {'title': 'Question', 'description': 'The math question to answer.', 'type': 'string'}}
    - output_schema: { "result": string }

- Python REPL: A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out wi

OutputParserException: Got invalid JSON object. Error: Unterminated string starting at: line 1 column 575 (char 574)

In [ ]:
a = """{
  "solvability_thought": "The task is to find the capital of a given country. We can use the Search tool to find the capital of the input country.",
  "solvable": true,
  "plan": "- Use the Search tool with the query "capital of {country}"\n- Extract the capital from the search results\n- Return the capital as the output",
  "reflection": "The plan is straightforward and should work for most countries. However, there might be cases where the search results are not clear or ambiguous. In such cases, we might need to refine the search query or use a different source of information.",
  "plan_score": 8
}"""
a[220:222]

'ca'

In [ ]:
for tool in tools:
    print(tool.args)

{'tool_input': {'type': 'string'}}
{'question': {'title': 'Question', 'description': 'The math question to answer.', 'type': 'string'}}
{'query': {'title': 'Query', 'type': 'string'}}
{'url': {'title': 'Url', 'type': 'string'}}
{'commands': {'title': 'Commands', 'description': 'List of shell commands to run. Deserialized using json.loads', 'anyOf': [{'type': 'string'}, {'type': 'array', 'items': {'type': 'string'}}]}}


In [ ]:
import json

text = """{  "solvability_thought": "The task is to find the capital of a given country. We can use the Search tool to find the capital of the input country.",  "solvable": true,  "plan": "- Use the Search tool with the query "capital of {country}"- Extract the capital from the search results- Return the capital as the output",  "reflection": "The plan is straightforward and should work for most countries. However, there might be cases where the search results are not clear or ambiguous. In such cases, we might need to refine the search query or use a different source of information.",  "plan_score": 8}"""

try:
    json.loads(text)
except json.JSONDecodeError as e:
    if e.msg.startswith("Expecting ',' delimiter"):
        l_idx = e.pos - 1
        r_idx = l_idx + 1

        while text[r_idx] != '"':
            r_idx += 1

        print(json.loads(text[:l_idx] + '\\"' + text[l_idx+1:r_idx] + '\\"' + text[r_idx+1:]))
    else:
        print(e)

{'solvability_thought': 'The task is to find the capital of a given country. We can use the Search tool to find the capital of the input country.', 'solvable': True, 'plan': '- Use the Search tool with the query "capital of {country}"- Extract the capital from the search results- Return the capital as the output', 'reflection': 'The plan is straightforward and should work for most countries. However, there might be cases where the search results are not clear or ambiguous. In such cases, we might need to refine the search query or use a different source of information.', 'plan_score': 8}


In [ ]:
from typing import Optional, Type, List
# Import things that are needed generically
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from langchain.tools import Tool
from pydantic import BaseModel, Field

class CalculatorInput(BaseModel):
    question: List[str] = Field(description="The math question to answer.")
    
class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer questions about current events"

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        """Use the tool."""
        return search.run(query)
    
    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")
    
class CustomCalculatorTool(BaseTool):
    name = "Calculator"
    description = "useful for when you need to answer questions about math"
    args_schema: Type[BaseModel] = CalculatorInput

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        """Use the tool."""
        return llm_math_chain.run(query)
    
    async def _arun(self, query: str,  run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("Calculator does not support async")

In [ ]:
CustomSearchTool().args

{'query': {'title': 'Query', 'type': 'string'}}

In [ ]:
from inspect import signature

@tool
def f(a) -> List[int]:
    "AAA"
    pass

signature(f).return_annotation

'str'